In [3]:
import pandas as pd
import numpy as np
import dataImport as dataI
import random

In [4]:
train = pd.read_csv('../Data/youtube.csv')
train.head(20)

,Outlook,temps,humedity,windy,hours
0,1,3,1,0,25
1,1,3,1,1,30
2,2,3,1,0,46
3,3,2,1,0,45
4,3,1,0,0,52
5,3,1,0,1,23
6,2,1,0,1,43
7,1,2,1,0,35
8,1,1,0,0,38
9,3,2,0,0,46


In [49]:
df = train.loc[(train['temps']>=2) & (train['temps']<=3)]
# df = train.ix[2:3]

df


,Outlook,temps,humedity,windy,hours
0,1,3,1,0,25
1,1,3,1,1,30
2,2,3,1,0,46
3,3,2,1,0,45
7,1,2,1,0,35
9,3,2,0,0,46
10,1,2,0,1,48
11,2,2,1,1,52
12,2,3,0,0,44
13,3,2,1,1,30


In [51]:
df = train.ix[0:0]
df.Outlook.unique()[0]

1

In [14]:
def std_value(df,column_name,value,target):
    # Retorna el valor del desvio standar de una columna contra la de target
    std = np.std(df.loc[train[column_name]==value,target].values.flatten())
    len = df[column_name].count()
    len_value = df.loc[train[column_name]==value,target].count()
    prob = float(len_value)/len
    return std*prob

In [7]:
def calculate_std(df,column_name,target):
    # calcula los valores de desvio standar para cada columna
    stds = 0
    unique_values = df[column_name].unique()
    for value in unique_values:
        stds +=  std_value(df,column_name,value,target)
    return stds

In [8]:
def calculate_stds(df,columns_name,target):
    stds=[]
    for column_name in columns_name:
        stds.append((column_name,calculate_std(df,column_name,target)))
    return stds
        

In [9]:
def final_stds(stds_columns,target_std):
    stds=[]
    for tupple in stds_columns:
        stds.append((tupple[0],target_std - tupple[1]))
    return stds

In [10]:
def get_nrandom_columns(df,target,n_columns):
    columns_name = df.columns.tolist()
    columns_name.remove(target)
    return [ columns_name[i] for i in sorted(random.sample(xrange(len(columns_name)), n_columns)) ]

In [12]:
def get_split(df,target,n_columns):
    # Calculo std de la variable a predecir
    target_std = np.std(df.ix[:, target].tolist())
    
    columns_name = get_nrandom_columns(df,target,n_columns)
    
    # Calculamos la desviacion standard de cada columna:
    # Esto es: la sumatoria de (la probabilidad de cada uno de los distintos valores
    # de una columna * la desviacion standar del mismo) 
    stds_columns = calculate_stds(df,columns_name,target)
    
    # Restamos cada std de columna al std del target
    stds_columns = final_stds(stds_columns,target_std)

    # Obtengo el maximo valor
    max_std_column_name = max(stds_columns,key=lambda item:item[1])[0]
    
    # Devolvemos el nombre de la columna con mayor diferencia de std y una lista de listas
    # con los nodos hijos
    
    

    return (max_std_column_name,sub

In [ ]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [ ]:


def split(root, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

In [19]:
def build_tree(df, target, n_columns, max_depth, min_size):
    # target = nombre de la columna a predecir
    # n_columns =  cantidad de columnas random a considerar en cada split
    # n_columns < len(df.columns)!!!
    # para bagging n_columns = len(df.columns) - 1

    root = get_split(df, target, n_columns)
    tree = split(root, max_depth, min_size, 1)
    return tree
        
build_tree(train,'hours',(len(train.columns)-1)) #esto es para bagging

[('Outlook', 1.6621503366302335), ('temps', 0.47969057476332111), ('humedity', 0.27229619548982598), ('windy', 0.28214938055733185)]


'Outlook'